### Random forest did quite well, 94% under cross-validation.  I also tried XGBoost, which unsurprisingly did better, nearly 99% cross-validated, once I tuned it.
### I know I could do a bunch of visualization here, but again I was focused on prediction, on getting more practice with actual machine learning.  Didn't end up using GridSearch as it hardly seemed necessary.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectKBest

%matplotlib inline

In [2]:
df = pd.read_csv('Arrivals2000.csv')

In [3]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2000,1,28,5,1647.0,1647,1906.0,1859,HP,154,...,15,11,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2000,1,29,6,1648.0,1647,1939.0,1859,HP,154,...,5,47,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2000,1,30,7,NaN,1647,NaN,1859,HP,154,...,0,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2000,1,31,1,1645.0,1647,1852.0,1859,HP,154,...,7,14,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2000,1,1,6,842.0,846,1057.0,1101,HP,609,...,3,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN


### Start cleaning the data: remove Year because it's all the same.  Flight numbers are also irrelevant (if the flight number is relevant, it's because of other factors which are elsewhere, such as carrier incompetence, diversion, consistently bad weather between 2 locations, etc.).  Also, I'm removing cancellation data; being late or cancelled are mutually exclusive.
### I had been going to remove Tail Number, but with further thought realized that there could be some aircraft that are simply slower; they might be older, or prone to malfunction, who knows?  The way to identify them would be via the tail number, so it might be somewhat predictive.

In [4]:
# only keep flights that weren't cancelled
df = df[df['Cancelled'] == 0]

In [5]:
df.drop(['Year', 'DayofMonth', 'FlightNum', 'Cancelled', 'CancellationCode'], 1, inplace=True)

In [6]:
df.rename(columns={'DayOfWeek':'DWeek', 'UniqueCarrier':'CarrID', 'CRSElapsedTime':'SchElapTime',
                   'FlightNum':'Flight', 'CRSArrTime':'SchArr', 'CRSDepTime':'SchDep'}, inplace=True)

In [7]:
df.isnull().sum()

Month                      0
DWeek                      0
DepTime                    0
SchDep                     0
ArrTime                14254
SchArr                     0
CarrID                     0
TailNum                    0
ActualElapsedTime      14254
SchElapTime               10
AirTime                14254
ArrDelay               14254
DepDelay                   0
Origin                     0
Dest                       0
Distance                   0
TaxiIn                     0
TaxiOut                    0
Diverted                   0
CarrierDelay         5495557
WeatherDelay         5495557
NASDelay             5495557
SecurityDelay        5495557
LateAircraftDelay    5495557
dtype: int64

### All of the 'X'Delay columns contain only null values, so all are useless.

In [8]:
df.drop(['CarrierDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'WeatherDelay'], 1, inplace=True)

In [9]:
# Do we need these floats?  Floating point arithmetic takes more resources...
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5495557 entries, 0 to 5683046
Data columns (total 19 columns):
Month                int64
DWeek                int64
DepTime              float64
SchDep               int64
ArrTime              float64
SchArr               int64
CarrID               object
TailNum              object
ActualElapsedTime    float64
SchElapTime          float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               int64
TaxiOut              int64
Diverted             int64
dtypes: float64(7), int64(8), object(4)
memory usage: 838.6+ MB


### Only losing 14k rows by dropping NaN's

In [10]:
curr_size = df.shape[0]
no_nulls_size = df.dropna().shape[0]
print("Out of " + str(curr_size) + " rows, " + str(curr_size - no_nulls_size) + " will be lost by dropping NaN's")
df.dropna(inplace=True)

Out of 5495557 rows, 14254 will be lost by dropping NaN's


### I've looked through the data, and all these floating point numbers are functioning as integers (none seem to have decimal places actually being used).  I'm converting them for performance.  Moreover, there's no reason to use 64-bit integers, where 16-bit will suffice, saving space and computation.  There aren't any really big numbers in this data.

In [11]:
cols = df.select_dtypes(['float64', 'int64']).columns

for col in cols:
    df[col] = df[col].astype('int16')

In [12]:
# Well look at that, no diversions at all.  Drop this column.
df.loc[df['Diverted'] == 1]

,Month,DWeek,DepTime,SchDep,ArrTime,SchArr,CarrID,TailNum,ActualElapsedTime,SchElapTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Diverted


In [13]:
df.drop('Diverted', 1, inplace=True)
df.head()

,Month,DWeek,DepTime,SchDep,ArrTime,SchArr,CarrID,TailNum,ActualElapsedTime,SchElapTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,5,1647,1647,1906,1859,HP,N808AW,259,252,233,7,0,ATL,PHX,1587,15,11
1,1,6,1648,1647,1939,1859,HP,N653AW,291,252,239,40,1,ATL,PHX,1587,5,47
3,1,1,1645,1647,1852,1859,HP,N806AW,247,252,226,-7,-2,ATL,PHX,1587,7,14
4,1,6,842,846,1057,1101,HP,N158AW,255,255,244,-4,-4,ATL,PHX,1587,3,8
5,1,7,849,846,1148,1101,HP,N656AW,299,255,267,47,3,ATL,PHX,1587,8,24


### It looks like there are redundant columns, which will have very high correlation with one another.  For instance, ActualElapsedTime is simply AirTime, TaxiIn, and TaxiOut added together.  Also, I'm not sure how useful scheduled times are for prediction, since actual times are what predict being early/late (along with flight time).  Specifically, I'm looking at scheduled arrival and scheduled in-air times; Scheduled departure will certainly be useful as certain times of the day are more or less busy, and people work differently in the morning vs. afternoon or evening.
### Departure time likewise is simply derived from adding the scheduled departure with the departure delay; arrival time is similar.

In [14]:
df.drop(['DepTime', 'SchArr', 'ActualElapsedTime', 'ArrTime'], 1, inplace=True)

In [15]:
df.head()

,Month,DWeek,SchDep,CarrID,TailNum,SchElapTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,5,1647,HP,N808AW,252,233,7,0,ATL,PHX,1587,15,11
1,1,6,1647,HP,N653AW,252,239,40,1,ATL,PHX,1587,5,47
3,1,1,1647,HP,N806AW,252,226,-7,-2,ATL,PHX,1587,7,14
4,1,6,846,HP,N158AW,255,244,-4,-4,ATL,PHX,1587,3,8
5,1,7,846,HP,N656AW,255,267,47,3,ATL,PHX,1587,8,24


In [16]:
df.nunique()

Month            12
DWeek             7
SchDep         1183
CarrID           11
TailNum        4033
SchElapTime     497
AirTime         630
ArrDelay       1051
DepDelay       1023
Origin          206
Dest            206
Distance       1108
TaxiIn          181
TaxiOut         344
dtype: int64

### Convert categorical variables

In [17]:
convert = ['CarrID', 'Origin', 'Dest', 'TailNum']
le = LabelEncoder()
for name in convert:
    df[name] = le.fit_transform(df[name])

In [18]:
df.head()

,Month,DWeek,SchDep,CarrID,TailNum,SchElapTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,5,1647,5,3254,252,233,7,0,8,154,1587,15,11
1,1,6,1647,5,2528,252,239,40,1,8,154,1587,5,47
3,1,1,1647,5,3244,252,226,-7,-2,8,154,1587,7,14
4,1,6,846,5,293,255,244,-4,-4,8,154,1587,3,8
5,1,7,846,5,2543,255,267,47,3,8,154,1587,8,24


### These times make me nervous, as models will look at them as if they're actually numbers.  I'm going to convert them to raw minutes, as other time-related columns are just minutes.

In [19]:
def get_last_digits(num, digits=2):
    """
    This converts a time to purely minutes.
    IN: a 'time' in the form 1647 or 832; it's dealing with a bad format
    OUT: an integer
    EX: get_last_ditigs(1647) = 1007
        get_last_digits(632)  = 392
    """
    return (num // 100)*60 + (num % 10**digits)


In [20]:
df['SchDep'] = df['SchDep'].apply(get_last_digits)

In [21]:
df.head()

,Month,DWeek,SchDep,CarrID,TailNum,SchElapTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,5,1007,5,3254,252,233,7,0,8,154,1587,15,11
1,1,6,1007,5,2528,252,239,40,1,8,154,1587,5,47
3,1,1,1007,5,3244,252,226,-7,-2,8,154,1587,7,14
4,1,6,526,5,293,255,244,-4,-4,8,154,1587,3,8
5,1,7,526,5,2543,255,267,47,3,8,154,1587,8,24


### Take a subsample for testing

In [40]:
train_samp = df.sample(100000)
y = train_samp['ArrDelay']
X = train_samp.drop('ArrDelay', 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)

### Do a run with a random forest; turned out to be quite good

In [41]:
rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [42]:
predictions = rfr.predict(X_test)
print('\nR-squared:')
print(rfr.score(X_test, y_test))
cross = cross_val_score(rfr, X_train, y_train, cv=5)
print(cross)
print("Cross Validation Mean: ", cross.mean())

0.9594914581469073
0.9594914581469073
[0.96410515 0.96363157 0.96279804 0.92755681 0.9038075 ]
Cross Validation Mean:  0.9443798150646103


In [46]:
rfr.feature_importances_

array([0.00271415, 0.00207422, 0.00418984, 0.00257026, 0.00368617,
       0.02492101, 0.02688932, 0.84187855, 0.00393132, 0.00359294,
       0.00378222, 0.01157274, 0.06819727])

In [47]:
selector = SelectKBest(k=13)
X_new = selector.fit_transform(X_train, y_train)
names = X.columns.values[selector.get_support()]
scores = selector.scores_[selector.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)

     Feat_names     F_Scores
7      DepDelay  1102.724673
12      TaxiOut    37.420912
5   SchElapTime    12.550797
10     Distance     9.971317
6       AirTime     8.952449
3        CarrID     5.672168
2        SchDep     5.422968
11       TaxiIn     4.597506
4       TailNum     1.457786
0         Month     1.360205
9          Dest     1.275390
8        Origin     1.137196
1         DWeek     0.904761


### Now let's see how XGBoost does

In [59]:
xgb = XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=220, colsample_bylevel=0.4)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.4,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=220,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [60]:
predictions = xgb.predict(X_test)
print('\nR-squared:')
print(xgb.score(X_test, y_test))
cross = cross_val_score(xgb, X_train, y_train, cv=5)
print(cross)
print("Accuracy: %.2f%% (%.2f%%)" % (cross.mean()*100, cross.std()*100))


R-squared:
0.9793779388735974
[0.98725181 0.98099217 0.98424895 0.94343124 0.95001126]
Accuracy: 96.92% (1.86%)


### Aaannd let's run it on the whole set.  I expect it to take a couple hours.

In [61]:
y = df['ArrDelay']
X = df.drop('ArrDelay', 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)

In [62]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.4,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=220,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### It did take a couple hours, and it did very well too!

In [63]:
predictions = xgb.predict(X_test)
print('\nR-squared:')
print(xgb.score(X_test, y_test))
cross = cross_val_score(xgb, X_train, y_train, cv=5)
print(cross)
print("Accuracy: %.2f%% (%.2f%%)" % (cross.mean()*100, cross.std()*100))


R-squared:
0.9899538291348515
[0.99053557 0.98896328 0.9891445  0.98988191 0.99063218]
Accuracy: 98.98% (0.07%)
